# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум 4. Работа с API ВКонтакте: собираем посты со стены, комментарии к постам и информацию пользователях

## Подготовка к работе

Загружаем модули и библиотеки, необходимые для работы:

In [1]:
import requests
import time
import pandas as pd

Для начала давайте посмотрим на документацию API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.
По [инструкции](https://allatambov.github.io/pypolit/vk-auth.pdf) мы можем получить доступ к API, создадим приложение и скопируем его ID, чтобы получить ссылку для токена доступа:

In [ ]:
# если нет Управление в меню во ВКонтакте
# https://vk.com/apps?act=manage

In [2]:
app_id = input("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

Enter your client id: 51539694
https://oauth.vk.com/authorize?client_id=51539694&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token


Переходим по ссылке выше и копируем ссылку (полностью!) из адресной строки:

In [3]:
full_link = input()

https://oauth.vk.com/blank.html#access_token=vk1.a.1quwjHh3NkHYm-xSN7yQGLuTKk57U4wnPZy_Q6bfPtghRGPySfkEjlTvm2ZIcbAhJjiAt6QYu2_LtZCIOliKqW0GP5enwhEIYnuu3VoeJhJO1Zq65dtxIbuB6-lEyNBz3uD07ITMtpeqEH0CWOtvu1-xjdk7LHwFnFjmjrZXAjS8dGLquKzHCpqg6K_cnjFx&expires_in=86400&user_id=20473269


Разбиваем полученную ссылку (сначала по `access_token=`, потом часть после `=` по `&`) и забираем токен в чистом виде:

In [4]:
token = full_link.split("access_token=")[1].split("&")[0]

In [5]:
token

'vk1.a.1quwjHh3NkHYm-xSN7yQGLuTKk57U4wnPZy_Q6bfPtghRGPySfkEjlTvm2ZIcbAhJjiAt6QYu2_LtZCIOliKqW0GP5enwhEIYnuu3VoeJhJO1Zq65dtxIbuB6-lEyNBz3uD07ITMtpeqEH0CWOtvu1-xjdk7LHwFnFjmjrZXAjS8dGLquKzHCpqg6K_cnjFx'

Ура! Теперь токен доступа у нас есть, всё готово к работе!

## Часть 1: выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества [скалодрома Rock Zona](https://vk.com/rzclimbing). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [6]:
v = "5.131"
main_wall = "https://api.vk.com/method/wall.get"
domain = "rzclimbing"

Функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [7]:
params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [8]:
req_wall = requests.get(main_wall, params = params_wall)

In [9]:
req_wall.url

'https://api.vk.com/method/wall.get?access_token=vk1.a.1quwjHh3NkHYm-xSN7yQGLuTKk57U4wnPZy_Q6bfPtghRGPySfkEjlTvm2ZIcbAhJjiAt6QYu2_LtZCIOliKqW0GP5enwhEIYnuu3VoeJhJO1Zq65dtxIbuB6-lEyNBz3uD07ITMtpeqEH0CWOtvu1-xjdk7LHwFnFjmjrZXAjS8dGLquKzHCpqg6K_cnjFx&domain=rzclimbing&count=100&v=5.131'

In [1]:
json_wall = req_wall.json()
json_wall

Извлечём из этого большого словаря элемент, который отвечает за общее число постов на стене:

In [11]:
nposts = json_wall['response']['count']
print(nposts)

1461


Теперь извлечём элемент, который хранит результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [12]:
items_wall = json_wall['response']['items']

Посмотрим на один элемент такого списка:

In [13]:
i = items_wall[0]
i

{'id': 3873,
 'from_id': -38936316,
 'owner_id': -38936316,
 'date': 1675172057,
 'marked_as_ads': 0,
 'donut': {'is_donut': False},
 'comments': {'can_post': 1, 'count': 0, 'groups_can_post': True},
 'short_text_rate': 0.8,
 'carousel_offset': 0,
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1675172057,
    'id': 457243960,
    'owner_id': -38936316,
    'access_key': 'fe567bb2cf93da8b08',
    'sizes': [{'height': 75,
      'type': 's',
      'width': 75,
      'url': 'https://sun9-west.userapi.com/sun9-7/s/v1/ig2/dZd0_OuTD8d5hRpm8eAEGlkLrno7S9HQCgtBlg_rsJbteemkKN6cqUH03506YTW6DpePzVWfkaTOqiD13b32uWtS.jpg?size=75x75&quality=95&type=album'},
     {'height': 130,
      'type': 'm',
      'width': 130,
      'url': 'https://sun9-west.userapi.com/sun9-7/s/v1/ig2/pQBLen7JQUSkBM3AocvxHi2c4lS9G3Q_oLupcpMr0VIY2KlodCgFlErapTiTq0Ff4Omyehespj1LpbqRAxECKUms.jpg?size=130x130&quality=95&type=album'},
     {'height': 604,
      'type': 'x',
      'width': 604,
      'ur

### Задача 1

Извлеките из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [16]:
### YOUR CODE HERE ###
print(i["id"], i["date"], i["text"])
print(i["likes"]["count"], i["reposts"]["count"], 
     i["views"]["count"], i["comments"]["count"])

3873 1675172057 Фестиваль фестивалем, а новые трассы по расписанию! 🥳

Наши подготовщики добавили еще трасс на некоторые плоскости и полностью обновили одну стенку. 

💥 13 новых трасс ждут вас: 

2*5
3*6а/а+
3*6b
4*6c
1*7a

До встречи в Рок Зоне! 🤘
3 0 42 0


### Задача 2

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все. 

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [20]:
iterate = nposts // 100 + 1  # целочисленное деление + 1 

### Задача 3

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [21]:
params_wall_long = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v,
              "offset" : 0}

In [22]:
items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    # обновляем список items_all
    # обновляем парметр offset - увеличиваем на 100
    items_all.extend(items_wall_long)
    params_wall_long["offset"] = params_wall_long["offset"] + 100
    
    
    time.sleep(1.5)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


Проверяем длину списка – все ли посты собраны:

In [23]:
len(items_all)

1461

Не отбираем ничего на этом этапе, просто превращаем список словарей в датафрейм:

In [24]:
df = pd.DataFrame(items_all)
df.head(10)

,id,from_id,owner_id,date,marked_as_ads,donut,comments,short_text_rate,carousel_offset,attachments,...,post_type,reposts,text,views,hash,edited,copy_history,geo,signer_id,zoom_text
0,3873,-38936316,-38936316,1675172057,0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0.8,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",...,post,"{'count': 0, 'user_reposted': 0}","Фестиваль фестивалем, а новые трассы по распис...",{'count': 63},z0ich7XdEWLr1uU3qv-dulyxPrA,NaN,NaN,NaN,NaN,NaN
1,3872,-38936316,-38936316,1675080565,0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0.8,NaN,"[{'type': 'video', 'video': {'can_comment': 1,...",...,post,"{'count': 7, 'user_reposted': 0}",,{'count': 477},opsIVrtiyrkEJqmeyBi4ZmZmQhg,NaN,NaN,NaN,NaN,NaN
2,3871,-38936316,-38936316,1675014135,0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0.8,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",...,post,"{'count': 3, 'user_reposted': 0}",Наш фестиваль Boulderhouse фест подошел к конц...,{'count': 585},27H3CXvczTvf3Y7xVGlfG6mj3Ng,NaN,NaN,NaN,NaN,NaN
3,3870,-38936316,-38936316,1674989204,0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0.8,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",...,post,"{'count': 0, 'user_reposted': 0}",❓ Почему не улучшается растяжка\n\nВы делаете ...,{'count': 812},wPiFLkmrd3H-_iKtkWjCcS9nEuM,NaN,NaN,NaN,NaN,NaN
4,3868,-38936316,-38936316,1674902631,0,{'is_donut': False},"{'can_post': 1, 'count': 1, 'groups_can_post':...",0.8,NaN,"[{'type': 'video', 'video': {'can_comment': 1,...",...,post,"{'count': 3, 'user_reposted': 0}",,{'count': 663},KMvQ4AZUVjJylU2oL_3YPOofeUs,NaN,NaN,NaN,NaN,NaN
5,3867,-38936316,-38936316,1674744304,0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0.8,NaN,"[{'type': 'video', 'video': {'can_comment': 1,...",...,post,"{'count': 1, 'user_reposted': 0}",,{'count': 559},ewg73eWZ4BOWwOSl1RSenFXuqmI,NaN,NaN,NaN,NaN,NaN
6,3866,-38936316,-38936316,1674579500,0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0.8,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",...,post,"{'count': 5, 'user_reposted': 0}",❓ Как достигать целей\n\nБывало ли с вами тако...,{'count': 1706},X_KjtuKIHRDsk2YE7vWoEIXI1KI,NaN,NaN,NaN,NaN,NaN
7,3865,-38936316,-38936316,1674543053,0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0.8,NaN,"[{'type': 'video', 'video': {'can_comment': 1,...",...,post,"{'count': 0, 'user_reposted': 0}",,{'count': 493},4ZffB2U134MuuSAJ2s3_NY_WquI,NaN,NaN,NaN,NaN,NaN
8,3864,-38936316,-38936316,1674487494,0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0.8,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",...,post,"{'count': 0, 'user_reposted': 0}",🔥 Наш недельный фестиваль стартовал! Первая па...,{'count': 671},pKejCSlWU23L8iaoBiHGLN26N5g,NaN,NaN,NaN,NaN,NaN
9,3863,-38936316,-38936316,1674254432,0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0.8,NaN,"[{'type': 'doc', 'doc': {'id': 655860336, 'own...",...,post,"{'count': 1, 'user_reposted': 0}",Положение фестиваля BOULDERHOUSE FEST.,{'count': 580},LxnOku94afpcRK2YM7IvskMzL7Y,NaN,NaN,NaN,NaN,NaN


### Задача 4

Создайте на основе датафрейма `df` новый датафрейм `small` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

**Подсказка:** отберите сначала все столбцы с нужной информацией, напишите функцию для извлечения только числа лайков/просмотров/комментариев и примените её, удалите лишние столбцы.

In [30]:
# выбираем нужные столбцы
small = df[["id", "date", "text", "likes", "reposts", "comments"]]

In [31]:
# нужно научиться работать с ячейками такого вида
small["likes"][0]

{'can_like': 1, 'count': 3, 'user_likes': 0, 'can_publish': 1}

In [32]:
# пишем lambda-функцию, которая принимает на вход ячейку (x)
# и забирает оттуда значение по ключу count

small["nlikes"] = small["likes"].apply(lambda x: x["count"])
small["nreposts"] = small["reposts"].apply(lambda x: x["count"])
small["ncomments"] = small["comments"].apply(lambda x: x["count"])

<ipython-input-32-1d647d8670ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small["nlikes"] = small["likes"].apply(lambda x: x["count"])
<ipython-input-32-1d647d8670ed>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small["nreposts"] = small["reposts"].apply(lambda x: x["count"])
<ipython-input-32-1d647d8670ed>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [35]:
# удаляем лишние столбцы
# те, откуда извлекли count, больше не нужны
# inplace = True: сохраняем изменения сразу в small

small.drop(columns = ["reposts", "comments", "likes"], inplace =True)

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [36]:
small.head()

,id,date,text,nlikes,nreposts,ncomments
0,3873,1675172057,"Фестиваль фестивалем, а новые трассы по распис...",3,0,0
1,3872,1675080565,,27,7,0
2,3871,1675014135,Наш фестиваль Boulderhouse фест подошел к конц...,14,3,0
3,3870,1674989204,❓ Почему не улучшается растяжка\n\nВы делаете ...,10,0,0
4,3868,1674902631,,27,3,1


### Задача 5

Выберите только те строки в полученном датафрейме, которые соответствуют постам с числом комментариев больше 0, и сохраните их в датафрейм `with_comm`.

In [41]:
with_comm = small[small["ncomments"] > 0] 
with_comm.shape

(481, 6)

## Часть 2: собираем комментарии к постам

В целях экономии времени не будем брать весь датафрейм `with_comm`, для примера отберем первые 10 строк:

In [42]:
with_comm10 = with_comm.head(10) 

Заберем id постов и преобразуем их в список:

In [43]:
ids = list(with_comm10["id"])

Возьмем один id для примера и сформулируем запрос для получения комментариев к посту с этим id:

In [44]:
i = 3782

# метод wall.getComments
# owner_id: id сообщества
# post_id: id поста
# v: версия API, все та же
# access_token: токен доступа, все тот же
# count: число комментариев, берем 100, max возможное за раз
# thread_items_count: число ответов на комментарий, 10, max за раз

main_comm = "https://api.vk.com/method/wall.getComments"
params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}

Формулируем запрос, отправляем и смотрим на результат в виде словаря:

In [45]:
req = requests.get(main_comm, params = params_comm)
req.json() 

{'response': {'count': 4,
  'items': [{'id': 3783,
    'from_id': 527923327,
    'date': 1669311085,
    'text': 'Мотивация не нужна, нужна дисциплина.',
    'post_id': 3782,
    'owner_id': -38936316,
    'parents_stack': [],
    'thread': {'count': 1,
     'items': [{'id': 3784,
       'from_id': 2311030,
       'date': 1669314785,
       'text': '[id527923327|Николай], 👍',
       'post_id': 3782,
       'owner_id': -38936316,
       'parents_stack': [3783],
       'reply_to_user': 527923327,
       'reply_to_comment': 3783}],
     'can_post': True,
     'show_reply_button': True,
     'groups_can_post': True}},
   {'id': 3785,
    'from_id': 2847724,
    'date': 1669373425,
    'text': 'Мотивация тоже нужна)',
    'post_id': 3782,
    'owner_id': -38936316,
    'parents_stack': [],
    'thread': {'count': 1,
     'items': [{'id': 3788,
       'from_id': 527923327,
       'date': 1669400413,
       'text': '[id2847724|Татьяна], мотивация лишь временный фактор, дисциплина - вечна. Дис

### Задача 6

Давайте считать, что 100 самых новых комментариев и 10 ответов на них нам достаточно (если нет – аргумент `offset` есть, задача аналогична предыдущей по сбору постов). Повторите эту операцию выше для всех id в списке `ids` и сформируйте список с результатами. Преобразуйте результат в датафрейм `df_comm`. 

In [46]:
# по аналогии с циклом ранее, только ничего не увеличиваем
# подставляем на место post_id значение i

comments_all = []

for i in ids:
    params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}
    req = requests.get(main_comm, params = params_comm)
    comm_json = req.json() 
    comm_list = comm_json["response"]["items"]
    comments_all.extend(comm_list)
    time.sleep(1)

In [49]:
df_comm = pd.DataFrame(comments_all)

In [51]:
# смотрим на названия столбцов
# здесь id – id поста

with_comm10.head(2)

,id,date,text,nlikes,nreposts,ncomments
4,3868,1674902631,,27,3,1
12,3858,1674146111,,22,1,2


In [52]:
# смотрим на названия столбцов
# здесь id – id комментария, а post_id - id поста

df_comm.head(2)

,id,from_id,date,text,post_id,owner_id,parents_stack,thread,attachments
0,3869,196636514,1674936716,Не скручивайте 6-ку по красным зацепкам на сте...,3868,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
1,3859,185384079,1674148446,,3858,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...","[{'type': 'sticker', 'sticker': {'sticker_id':..."


### Задача 7

Объедините датафрейм с собранными комментариями `df_comm` и датафрейм `with_comm10` через функцию `merge()` из `pandas`.

In [53]:
# переименовываем столбцы в with_comm10, чтобы 
# столбец с id поста назывался везде одинаково
# inplace = True: сохраняем изменения

with_comm10.rename(columns = {"id" : "post_id"}, inplace = True)
with_comm10.head(2)

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,post_id,date,text,nlikes,nreposts,ncomments
4,3868,1674902631,,27,3,1
12,3858,1674146111,,22,1,2


In [56]:
# объединяем with_comm10 и df_comm по столбцу post_id
# left: к левому датафрейму (with_comm10) подтягиваем данные из правого (df_comm)

final = with_comm10.merge(df_comm, on = "post_id", how = "left")

In [57]:
final

,post_id,date_x,text_x,nlikes,nreposts,ncomments,id,from_id,date_y,text_y,owner_id,parents_stack,thread,attachments
0,3868,1674902631,,27,3,1,3869,196636514,1674936716,Не скручивайте 6-ку по красным зацепкам на сте...,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
1,3858,1674146111,,22,1,2,3859,185384079,1674148446,,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...","[{'type': 'sticker', 'sticker': {'sticker_id':..."
2,3858,1674146111,,22,1,2,3860,41414377,1674157341,Пора полный метр снимать👍,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
3,3856,1674042051,Кампус и фингерборд — отличные инструменты для...,30,11,1,3857,-38936316,1674042052,#скалодорммосква #скалодром #скалолазаниевмоск...,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
4,3851,1673619321,,16,1,1,3852,94783194,1673619998,))),-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
5,3847,1673273731,Как втянуться в тренировочный режим после праз...,31,4,1,3848,260091507,1673504662,У меня есть альтернативное предложение: отказа...,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
6,3838,1672414128,В эту среду у нас на скалодроме прошел новогод...,19,1,3,3839,260091507,1672414774,"Спасибо, Рокзона, вечеринка вполне удалась!",-38936316,[],"{'count': 1, 'items': [{'id': 3841, 'from_id':...",NaN
7,3838,1672414128,В эту среду у нас на скалодроме прошел новогод...,19,1,3,3840,-38936316,1672416207,Полные результаты \n\nЛюбители :\n\n1.Гончаров...,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
8,3832,1672160535,"Скалолазы часто путают эти два термина, потому...",42,10,5,3833,642284329,1672205924,"Мощность по-вашему ещё называется ""взрывная си...",-38936316,[],"{'count': 1, 'items': [{'id': 3836, 'from_id':...",NaN
9,3832,1672160535,"Скалолазы часто путают эти два термина, потому...",42,10,5,3843,2335439,1672679368,"Мощноть это работа за время, а также сила на с...",-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN


## Часть 3:  собираем информацию о пользователях

Для примера выберем пользователя с id 20473269 (пока не из датафрейма, это всего лишь я):

In [ ]:
j = 20473269

Сформируем запрос:

In [ ]:
# метод users.get
# все знакомые параметры
# в fields перечисляем дополнительню информацию,
# которую хотим получить
# перечисляем через запятую без пробелов

main_user = "https://api.vk.com/method/users.get"
params_user = {"access_token" : token,
               "v" : v,
               "user_id" : j,
               "fields" : "bdate,city,home_town,universities"
}
req2 = requests.get(main_user, params = params_user)
req2.json()

### Задача 8

Выберите какого-нибудь пользователя, оставившего комментарий, из датафрейма и запросите по нему интересующую информацию.

In [ ]:
### YOUR CODE HERE ###